# Initialization

## Import Libraries

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

# Fast AI (PyTorch wrapper)
from fastai import *
from fastai.vision.all import *
import fastai
fastai.__version__

'2.5.3'

In [2]:
from collections import namedtuple
import tqdm

In [3]:
# make sure GPU is being used 
torch.cuda.current_device() 
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [4]:
# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

## Import Created Modules

In [5]:
from quantization_functions import quant_aware_resnet_model
from quantization_functions import post_training_quant_model
from quantization_functions import train_loop

## Load Dataset

In [6]:
BATCH_SIZE = 128
TEST_BATCH_SIZE = 32
N_CLASS = 10

In [7]:
# Download Imagenette 320 pixel

path = untar_data(URLs.IMAGENETTE_320)

In [8]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(112),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize(*imagenet_stats,inplace=True)
])

test_tfms = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(112),
    transforms.ToTensor(),
    transforms.Normalize(*imagenet_stats)
])

In [9]:
# PyTorch datasets

trainset = datasets.ImageFolder(path/"train", train_tfms)
testset = datasets.ImageFolder(path/"val", test_tfms)

# PyTorch data loaders

train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=2)

# Resnet 18 Models

In [10]:
N_EPOCH = 10

In [11]:
SAVE_DIR = 'checkpoint/imagenette_resnet18'

## Base model

In [12]:
base_model = torchvision.models.resnet18(pretrained=True)
base_model.fc = nn.Linear(base_model.fc.in_features, N_CLASS) # Change top layer

In [13]:
### Train Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(base_model.parameters(), 1e-2, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    clip_value=1e-2,
    epochs=N_EPOCH, save=f"{SAVE_DIR}/base_model"
)

train - epoch:  0: : 74it [01:02,  1.18it/s, loss=0.609]                      
val - epoch:  0: : 123it [00:16,  7.63it/s, val_loss=0.244, train_loss=0.609, acc=0.921]                       
train - epoch:  1: : 74it [00:27,  2.69it/s, loss=0.291]                      
val - epoch:  1: : 123it [00:12,  9.69it/s, val_loss=0.195, train_loss=0.291, acc=0.937]                       
train - epoch:  2: : 74it [00:27,  2.65it/s, loss=0.246]                      
val - epoch:  2: : 123it [00:13,  9.43it/s, val_loss=0.203, train_loss=0.246, acc=0.942]                       
train - epoch:  3: : 74it [00:27,  2.68it/s, loss=0.228]                      
val - epoch:  3: : 123it [00:12,  9.52it/s, val_loss=0.202, train_loss=0.228, acc=0.935]                       
train - epoch:  4: : 74it [00:28,  2.62it/s, loss=0.209]                      
val - epoch:  4: : 123it [00:12,  9.48it/s, val_loss=0.195, train_loss=0.209, acc=0.941]                       
train - epoch:  5: : 74it [00:27,  2.72it/s, 

In [16]:
base_model = torchvision.models.resnet18(pretrained=False)
base_model.fc = nn.Linear(base_model.fc.in_features, N_CLASS) # Change top layer

base_model.load_state_dict(torch.load(f'{SAVE_DIR}/base_model/model_weights.pt'))

# Validation accuracy
train_loop.test_model(test_loader, base_model)

100%|██████████| 123/123 [00:13<00:00,  9.16it/s, acc=0.946]

acc: 0.9464968152866242


## Post Training Quantization

### 8 bit quantization

In [12]:
# Convert base model to a custom quantization layer with the trained weights
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=8, qat=False,
                                                  pretrained=f'{SAVE_DIR}/base_model/model_weights.pt')
c_base_model.quantize(True)

remained state dict odict_keys([])


In [13]:
# Forward pass to have quantized weights
train_loop.test_model(train_loader, c_base_model)

100%|██████████| 74/74 [00:26<00:00,  2.75it/s, acc=0.946]

acc: 0.9455063892702503


In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:13<00:00,  9.25it/s, acc=0.944]

acc: 0.9439490445859873


In [15]:
with open(f'{SAVE_DIR}/ptq8bit_model_weights.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 7-bit quantization

In [12]:
# Convert base model to a custom quantization layer with the trained weights
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=7, qat=False,
                                                  pretrained=f'{SAVE_DIR}/base_model/model_weights.pt')
c_base_model.quantize(True)

remained state dict odict_keys([])


In [13]:
# Forward pass to have quantized weights
train_loop.test_model(train_loader, c_base_model)

100%|██████████| 74/74 [00:28<00:00,  2.57it/s, acc=0.943]

acc: 0.9432886260428768


In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:13<00:00,  8.83it/s, acc=0.94] 

acc: 0.9403821656050956


In [16]:
with open(f'{SAVE_DIR}/ptq7bit_model_weights.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 6-bit quantization

In [12]:
# Convert base model to a custom quantization layer with the trained weights
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=6, qat=False,
                                                  pretrained=f'{SAVE_DIR}/base_model/model_weights.pt')
c_base_model.quantize(True)

remained state dict odict_keys([])


In [13]:
# Forward pass to have quantized weights
train_loop.test_model(train_loader, c_base_model)

100%|██████████| 74/74 [00:28<00:00,  2.60it/s, acc=0.946]

acc: 0.9455063892702503


In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:13<00:00,  9.07it/s, acc=0.903]

acc: 0.9026751592356688


In [16]:
with open(f'{SAVE_DIR}/ptq6bit_model_weights.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 5-bit quantization

In [12]:
# Convert base model to a custom quantization layer with the trained weights
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=5, qat=False,
                                                  pretrained=f'{SAVE_DIR}/base_model/model_weights.pt')
c_base_model.quantize(True)

remained state dict odict_keys([])


In [13]:
# Forward pass to have quantized weights
train_loop.test_model(train_loader, c_base_model)

100%|██████████| 74/74 [00:28<00:00,  2.55it/s, acc=0.946]

acc: 0.9459288203611785


In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:13<00:00,  8.92it/s, acc=0.255] 

acc: 0.255031847133758


In [16]:
with open(f'{SAVE_DIR}/ptq5bit_model_weights.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 4-bit quantization

In [12]:
# Convert base model to a custom quantization layer with the trained weights
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=4, qat=False,
                                                  pretrained=f'{SAVE_DIR}/base_model/model_weights.pt')
c_base_model.quantize(True)

remained state dict odict_keys([])


In [13]:
# Forward pass to have quantized weights
train_loop.test_model(train_loader, c_base_model)

100%|██████████| 74/74 [00:27<00:00,  2.67it/s, acc=0.947]

acc: 0.9465624669975711


In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:13<00:00,  9.00it/s, acc=0.128]  

acc: 0.12840764331210192


In [16]:
with open(f'{SAVE_DIR}/ptq4bit_model_weights.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

## Quantization Aware Training

### 8-bit quantization

In [12]:
# Create model with custom quantization layer from the start
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=8, qat=True, pretrained=True)
c_base_model.quantize(True)

remained state dict odict_keys(['fc.weight', 'fc.bias'])


In [13]:
# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    clip_value=1e-2,
    epochs=N_EPOCH, save=f"{SAVE_DIR}/qat8bit"
)

train - epoch:  0: : 74it [00:29,  2.52it/s, loss=0.886]                      
val - epoch:  0: : 123it [00:14,  8.22it/s, val_loss=0.282, train_loss=0.886, acc=0.907]                       
train - epoch:  1: : 74it [00:28,  2.57it/s, loss=0.393]                      
val - epoch:  1: : 123it [00:14,  8.60it/s, val_loss=0.246, train_loss=0.393, acc=0.913]                       
train - epoch:  2: : 74it [00:27,  2.69it/s, loss=0.328]                      
val - epoch:  2: : 123it [00:14,  8.34it/s, val_loss=0.221, train_loss=0.328, acc=0.929]                       
train - epoch:  3: : 74it [00:29,  2.53it/s, loss=0.293]                      
val - epoch:  3: : 123it [00:14,  8.62it/s, val_loss=0.21, train_loss=0.293, acc=0.934]                        
train - epoch:  4: : 74it [00:29,  2.55it/s, loss=0.283]                      
val - epoch:  4: : 123it [00:14,  8.38it/s, val_loss=0.203, train_loss=0.283, acc=0.935]                       
train - epoch:  5: : 74it [00:28,  2.60it/s, 

In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:13<00:00,  8.87it/s, acc=0.944]

acc: 0.9439490445859873


In [16]:
with open(f'{SAVE_DIR}/qat8bit/model_weights_quantized.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 7-bit quantization

In [12]:
# Create model with custom quantization layer from the start
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=7, qat=True, pretrained=True)
c_base_model.quantize(True)

remained state dict odict_keys(['fc.weight', 'fc.bias'])


In [13]:
# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    clip_value=1e-2,
    epochs=N_EPOCH, save=f"{SAVE_DIR}/qat7bit"
)

train - epoch:  0: : 74it [00:29,  2.50it/s, loss=0.982]                      
val - epoch:  0: : 123it [00:14,  8.32it/s, val_loss=0.342, train_loss=0.982, acc=0.887]                       
train - epoch:  1: : 74it [00:28,  2.58it/s, loss=0.395]                      
val - epoch:  1: : 123it [00:14,  8.43it/s, val_loss=0.272, train_loss=0.395, acc=0.911]                       
train - epoch:  2: : 74it [00:29,  2.54it/s, loss=0.355]                      
val - epoch:  2: : 123it [00:14,  8.42it/s, val_loss=0.23, train_loss=0.355, acc=0.928]                        
train - epoch:  3: : 74it [00:28,  2.60it/s, loss=0.331]                      
val - epoch:  3: : 123it [00:14,  8.24it/s, val_loss=0.216, train_loss=0.331, acc=0.93]                        
train - epoch:  4: : 74it [00:28,  2.61it/s, loss=0.297]                      
val - epoch:  4: : 123it [00:14,  8.46it/s, val_loss=0.21, train_loss=0.297, acc=0.933]                        
train - epoch:  5: : 74it [00:28,  2.63it/s, 

In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:14<00:00,  8.77it/s, acc=0.94] 

acc: 0.9401273885350319


In [16]:
with open(f'{SAVE_DIR}/qat7bit/model_weights_quantized.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 6-bit quantization

In [10]:
# Create model with custom quantization layer from the start
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=6, qat=True, pretrained=True)
c_base_model.quantize(True)

remained state dict odict_keys(['fc.weight', 'fc.bias'])


In [11]:
# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-2, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    clip_value=1e-2,
    epochs=N_EPOCH, save=f"{SAVE_DIR}/qat6bit"
)

train - epoch:  0: : 196it [00:25,  7.60it/s, loss=1.06]                       
val - epoch:  0: : 40it [00:03, 12.22it/s, val_loss=0.847, train_loss=1.06, acc=0.701]                      
train - epoch:  1: : 196it [00:26,  7.48it/s, loss=0.728]                       
val - epoch:  1: : 40it [00:03, 13.22it/s, val_loss=0.805, train_loss=0.728, acc=0.733]                      
train - epoch:  2: : 196it [00:25,  7.59it/s, loss=0.626]                       
val - epoch:  2: : 40it [00:02, 13.47it/s, val_loss=0.698, train_loss=0.626, acc=0.77]                       
train - epoch:  3: : 196it [00:25,  7.63it/s, loss=0.563]                       
val - epoch:  3: : 40it [00:02, 13.60it/s, val_loss=0.707, train_loss=0.563, acc=0.763]                      
train - epoch:  4: : 196it [00:25,  7.67it/s, loss=0.515]                       
val - epoch:  4: : 40it [00:02, 13.64it/s, val_loss=0.677, train_loss=0.515, acc=0.781]                      
train - epoch:  5: : 196it [00:25,  7.70it/s, l

In [12]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [13]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 40/40 [00:03<00:00, 10.79it/s, acc=0.769]

acc: 0.7689


In [14]:
with open(f'{SAVE_DIR}/qat6bit/model_weights_quantized.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 5-bit quantization

In [12]:
# Create model with custom quantization layer from the start
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=5, qat=True, pretrained=True)
c_base_model.quantize(True)

remained state dict odict_keys(['fc.weight', 'fc.bias'])


In [13]:
# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    clip_value=1e-2,
    epochs=N_EPOCH, save=f"{SAVE_DIR}/qat5bit"
)

train - epoch:  0: : 74it [00:28,  2.61it/s, loss=1.9]                       
val - epoch:  0: : 123it [00:14,  8.59it/s, val_loss=1.06, train_loss=1.9, acc=0.639]                       
train - epoch:  1: : 74it [00:28,  2.62it/s, loss=1.1]                       
val - epoch:  1: : 123it [00:14,  8.45it/s, val_loss=0.794, train_loss=1.1, acc=0.735]                       
train - epoch:  2: : 74it [00:28,  2.64it/s, loss=0.954]                      
val - epoch:  2: : 123it [00:15,  8.15it/s, val_loss=0.692, train_loss=0.954, acc=0.766]                       
train - epoch:  3: : 74it [00:29,  2.52it/s, loss=0.885]                      
val - epoch:  3: : 123it [00:14,  8.43it/s, val_loss=0.631, train_loss=0.885, acc=0.786]                       
train - epoch:  4: : 74it [00:28,  2.63it/s, loss=0.922]                      
val - epoch:  4: : 123it [00:14,  8.46it/s, val_loss=0.694, train_loss=0.922, acc=0.779]                       
train - epoch:  5: : 74it [00:28,  2.56it/s, loss=0.

In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:15<00:00,  7.99it/s, acc=0.806]

acc: 0.8061146496815287


In [16]:
with open(f'{SAVE_DIR}/qat5bit/model_weights_quantized.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)

### 4-bit quantization

In [12]:
# Create model with custom quantization layer from the start
c_base_model = quant_aware_resnet_model.CResnet18(num_class=10, q_num_bit=4, qat=True, pretrained=True)
c_base_model.quantize(True)

remained state dict odict_keys(['fc.weight', 'fc.bias'])


In [13]:
# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    clip_value=1e-2,
    epochs=N_EPOCH, save=f"{SAVE_DIR}/qat4bit"
)

train - epoch:  0: : 74it [00:30,  2.45it/s, loss=2.54]                      
val - epoch:  0: : 123it [00:14,  8.48it/s, val_loss=2.34, train_loss=2.54, acc=0.121]                       
train - epoch:  1: : 74it [00:29,  2.51it/s, loss=2.31]                      
val - epoch:  1: : 123it [00:14,  8.37it/s, val_loss=2.28, train_loss=2.31, acc=0.145]                       
train - epoch:  2: : 74it [00:29,  2.54it/s, loss=2.53]                      
val - epoch:  2: : 123it [00:14,  8.40it/s, val_loss=2.37, train_loss=2.53, acc=0.141]                       
train - epoch:  3: : 74it [00:28,  2.56it/s, loss=2.36]                      
val - epoch:  3: : 123it [00:14,  8.47it/s, val_loss=2.25, train_loss=2.36, acc=0.164]                       
train - epoch:  4: : 74it [00:27,  2.65it/s, loss=2.26]                      
val - epoch:  4: : 123it [00:14,  8.52it/s, val_loss=2.24, train_loss=2.26, acc=0.15]                        
train - epoch:  5: : 74it [00:30,  2.44it/s, loss=2.24]     

In [14]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet18(num_class=10)
q_base_model.convert_from(c_base_model)

In [15]:
# Validation accuracy
train_loop.test_model(test_loader, q_base_model)

100%|██████████| 123/123 [00:14<00:00,  8.51it/s, acc=0.133]

acc: 0.1332484076433121


In [16]:
with open(f'{SAVE_DIR}/qat4bit/model_weights_quantized.pt', 'wb') as f:
    torch.save(q_base_model.state_dict(), f)